# Dump salaries from 990EZ in OR (continuing from part 1)

In [1]:
import unicodecsv as csv
from irsx.xmlrunner import XMLRunner
import pandas as pd

In [ ]:
# filers = pd.read_csv("orefilers.csv")
# ez = filers["RETURN_TYPE"] == "990EZ"
# reader = filers[ez]
# reader.to_csv('orefilers_ez.csv',index=False)

In [43]:
# read the whole file of orgs with efilings from part 1 here, it's not very long
ez_rows = [] 
# We're using the output of part 1
with open('orefilers.csv', 'rb') as infile:
    reader = csv.DictReader(infile)
    for row in reader:
        if row['RETURN_TYPE'] == "990EZ" or row['RETURN_TYPE'] == "990EO":
            ez_rows.append(row['OBJECT_ID'])
ez_rows

['201841359349206934',
 '201803169349202580',
 '201801509349200315',
 '201841359349203229',
 '201803139349201610',
 '201813329349200711',
 '201930599349200918',
 '201803449349200110',
 '201811769349200046',
 '201833119349200808',
 '201713319349200006',
 '201813169349202996',
 '201823069349200862',
 '201801699349200745',
 '201843409349200109',
 '201842649349200124',
 '201813189349203431',
 '201833199349201723',
 '201713199349208761',
 '201832269349200643',
 '201811669349200506',
 '201823139349200962',
 '201801919349200850',
 '201842419349200729',
 '201803419349200515',
 '201833129349201573',
 '201822559349200332',
 '201833549349200763',
 '201803179349202185',
 '201833199349207858',
 '201833179349204063',
 '201812049349200831',
 '201930089349201018',
 '201803179349201080',
 '201713189349200131',
 '201823209349200722',
 '201822259349200332',
 '201822839349200037',
 '201900109349200525',
 '201813039349201166',
 '201823189349206357',
 '201813179349204216',
 '201911969349200631',
 '201931029

In [ ]:
# the name of the output file
outfilename ="ez_comp_1.csv"
outfile = open(outfilename , 'wb')

# the header rows as they'll appear in the output
headers = ["ein", "object_id", "exec_comp"]
# start up a dictwriter, ignore extra rows
dw = csv.DictWriter(outfile, headers, extrasaction='ignore')
dw.writeheader()

In [3]:
# get an XMLRunner -- this is what actually does the parsing
xml_runner = XMLRunner()

## Figure out what to extract

Data from each repeating group should go to it's own file, otherwise it won't make sense.

To figure out what to capture, I started by looking at schedule J: http://www.irsx.info/#IRS990ScheduleJ
Then I went to the table details and picked the rows I wanted from the repeating group:
http://www.irsx.info/metadata/groups/SkdJRltdOrgOffcrTrstKyEmpl.html

Note that it's common for director/employee names in schedule J to get listed as businessname.

Also note that IRSx checks to see if a file has been downloaded before fetching it. Running this the first time will be slow if the filings aren't already downloaded, but much faster if they've already been downloaded.

-------NOTE--------
Edited to get compensation from EZ partIV, not sked j


In [41]:
execs = {}
for object_id in file_rows:
    parsed_filing = xml_runner.run_filing(object_id)
    parsed_main = parsed_filing.get_parsed_sked('IRS990EZ')[0]
    employee_groups = parsed_main['groups']['EZOffcrDrctrTrstEmpl']
    exec_comp = 0
    for employee_group in employee_groups:

        comp = int(employee_group.get('CmpnstnAmt'))
        if comp>0:
            exec_comp += comp
        if employee_group.get('EmplyBnftPrgrmAmt'):
            exec_comp += int(employee_group.get('EmplyBnftPrgrmAmt'))
        if employee_group.get('ExpnsAccntOthrAllwncAmt'):
            exec_comp += int(employee_group.get('ExpnsAccntOthrAllwncAmt'))

    execs[object_id] = exec_comp

execs['201713319349200006']

17709

In [ ]:
DEMO_MAX = 40
num_rows = 0

all_rows = []

for row in file_rows:
    num_rows += 1
    this_object_id = row['OBJECT_ID']
    parsed_filing = xml_runner.run_filing(this_object_id)
    
    # if it somehow busted, just note it and continue
    if not parsed_filing:
        print("Skipping filing %s(filings with pre-2013 schemas are skipped)\n row details: %s" % (this_object_id, row))
        continue 
    
    schedule_list = parsed_filing.list_schedules()
    exec_comp = 0
    if 'IRS990EZ' in schedule_list:
        
        
        # store the output in this dict
        outputdata = {}
        # assign some initial values from the input csv
        outputdata['ein'] = row['EIN']
        outputdata['object_id'] = row['OBJECT_ID']
        
        parsed_main = parsed_filing.get_parsed_sked('IRS990EZ')[0] 
        
        # now use the table name we looked up -- that's where we find it
        try:
            # repeating groups are returned as an array of dicts
            employee_groups = parsed_main['groups']['EZOffcrDrctrTrstEmpl']
        except KeyError:
            print("No EZOffcrDrctrTrstEmpl found in %s skipping" % this_object_id)
            continue
          
        # read through each employee and pull out the data we want
        for employee_group in employee_groups:
            
            comp = int(employee_group.get('CmpnstnAmt'))
            if comp>0:
                exec_comp += comp
            if employee_group.get('EmplyBnftPrgrmAmt'):
                exec_comp += int(employee_group.get('EmplyBnftPrgrmAmt'))
            if employee_group.get('ExpnsAccntOthrAllwncAmt'):
                exec_comp += int(employee_group.get('ExpnsAccntOthrAllwncAmt'))

        outputdata['exec_comp'] = exec_comp
        all_rows.append(outputdata)
#         dw.writerow(outputdata)
        

# Don't run endlessly during a demo:
#     if(num_rows > DEMO_MAX):
#        break

#     if num_rows%100==0:
#         print("Processed %s filings" % num_rows)
        
# close the outfile
# outfile.close()

# print("end")

In [ ]:
s = next((item for item in all_rows if item["ein"] == "814325149"), False)
print(s['exec_comp'])